# Reflection Agent

## Workflow of Reflection Agent in LangGraph:
1. Generation Node: Create Initial Draft
The generation node quickly produces a rough first draft based on the prompt. It focuses on speed over perfection. For example, generating a basic outline for a YouTube video which is then passed along for review.

2. Evaluation Node: Assess Quality
The evaluation node checks if the draft meets key criteria like clarity, relevance, and tone. If the content is good enough (e.g., a YouTube video that feels engaging and on-topic), it moves to the final stage; otherwise, it needs refinement.

3. Reflection Node: Refine and Improve
If improvements are needed, the reflection node critiques and revises the draft. It fine-tunes tone, adds clarity, and enhances engagement until the content reaches the desired quality.

4. Final Output: Deliver Polished Result
After reflection, the process produces a final, high-quality version — for instance, a well-crafted YouTube video script — ready to be published or delivered to the user.

![Refection Agent](../.github/assets/reflection_agent.png)

In [13]:
from langchain_core.messages import BaseMessage
from langchain.chat_models import init_chat_model

from typing import Sequence
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [14]:
llm = init_chat_model("llama3.1", model_provider="ollama")

## Generation Chain

In [15]:
generation_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a professional X (formally known as twitter) content assistant tasked with crafting engaging, insightful, and well-structured X (formally known as twitter) posts."
            " Generate the best X (formally known as twitter) post possible for the user's request."
            " Never return more then exactly 1 tweet at a time"
            " If the user provides feedback or critique, respond with a refined version of your previous attempts, improving clarity, tone, or engagement as needed.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [16]:
generate_chain = generation_prompt | llm

## Reflection Chain

In [17]:
reflection_prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        """You are a professional X (formally twitter) content strategist and thought leadership expert. Your task is to critically evaluate the given X (formally twitter) post and provide a comprehensive critique. Follow these guidelines:

        1. Assess the post’s overall quality, professionalism, and alignment with X (formally twitter) best practices.
        2. Evaluate the structure, tone, clarity, and readability of the post.
        3. Analyze the post’s potential for engagement (likes, comments, shares) and its effectiveness in getting engagement.
        4. Consider the post’s relevance to the topic, audience, or current trends.
        5. Examine the use of formatting (e.g., line breaks, bullet points), hashtags, mentions, and media (if any).
        6. Evaluate the effectiveness of any call-to-action or takeaway.

        Provide a detailed critique that includes:
        - A brief explanation of the post’s strengths and weaknesses.
        - Specific areas that could be improved.
        - Actionable suggestions for enhancing clarity, engagement, and professionalism.

        Your critique will be used to improve the post in the next revision step, so ensure your feedback is thoughtful, constructive, and practical.
        """
    ),
    MessagesPlaceholder(variable_name="messages")
])

In [18]:
reflect_chain = reflection_prompt | llm

## Building the Agent Graph

In [19]:
from langgraph.graph import END, MessageGraph, StateGraph
from typing import List, TypedDict
from langchain.schema import HumanMessage, AIMessage



In [20]:
def generation_node(state: Sequence[BaseMessage]) -> List[BaseMessage]:
    generated_post = generate_chain.invoke({"messages": state})
    return [AIMessage(content=generated_post.content)]

In [21]:
def reflection_node(messages: Sequence[BaseMessage]) -> List[BaseMessage]:
    res = reflect_chain.invoke({"messages": messages})  # Passes messages as input to reflect_chain
    return [HumanMessage(content=res.content)]  # Returns the refined message as HumanMessage for feedback

In [22]:
# Initialize a predefined MessageGraph
graph = MessageGraph()
graph.add_node("generate", generation_node)
graph.add_node("reflect", reflection_node)
graph.add_edge("reflect", "generate")
graph.set_entry_point("generate")

def should_continue(state: List[BaseMessage]):
    if len(state) > 6:
        return END
    return "reflect"

graph.add_conditional_edges("generate", should_continue)

workflow = graph.compile()

In [23]:
inputs = HumanMessage(content="""Write a tweet about the potential of AI Agents.""")

In [24]:
response = workflow.invoke(inputs)

In [25]:
for resp in response:
    print(resp.content)

Write a tweet about the potential of AI Agents.
"AI agents are no longer just futuristic concepts - they're here & changing the game! From personalized assistants to autonomous decision-makers, their potential is vast & unprecedented. What possibilities will they unlock for humanity next? #AIAgent #FutureOfWork"
 

**Comprehensive Critique:**

1. **Overall Quality and Professionalism:** The tweet demonstrates a good understanding of AI Agents' capabilities and potential impact. However, it could benefit from more nuance and specificity to make the topic feel more relatable.
2. **Structure, Tone, Clarity, and Readability:** The tweet is concise, but its ideas are somewhat loosely connected. A clearer thesis or central argument would strengthen the narrative.
3. **Engagement Potential:** While the question "What possibilities will they unlock for humanity next?" encourages speculation and engagement, it might be more effective to ground this in a concrete example or personal experience.


In [26]:
print(response[-1].content)

"Benchmarking AI agents' impact: a study shows 80% of businesses see significant efficiency gains with AI-powered customer support. Time to rethink the future of work? What role will humans play alongside AI in industries like retail, finance & more? Share your thoughts! #AIAgent #FutureOfWork"


# QA-WebSearch Agent

In [27]:
from typing import Optional

In [28]:
# Define the structure of the QA state
class QAState(TypedDict):
    # 'question' stores the user's input question. It can be a string or None if not provided.
    question: Optional[str]
    
    # 'context' stores relevant context about the guided project, if the question pertains to it.
    # If the question isn't related to the project, this will be None.
    context: Optional[str]
    
    # 'answer' stores the generated response or answer. It can be None until the answer is generated.
    answer: Optional[str]

In [29]:
def input_validation_node(state):
    # Extract the question from the state, and strip any leading or trailing spaces
    question = state.get("question", "").strip()
    
    # If the question is empty, return an error message indicating invalid input
    if not question:
        return {"valid": False, "error": "Question cannot be empty."}
    
    # If the question is valid, return valid status
    return {"valid": True}

In [30]:
from langchain.tools import Tool
from langchain.prompts import PromptTemplate
from ddgs import DDGS

import requests
from bs4 import BeautifulSoup

def fetch_and_clean(url):
    try:
        r = requests.get(url, timeout=10)
        soup = BeautifulSoup(r.text, "html.parser")
        return soup.get_text()
    except Exception:
        return ""

def duckduckgo_extended(query: str, max_results=3) -> list:
    summary_prompt = PromptTemplate.from_template(
        "Summarize the content in 5-8 sentences relevant to the query."
        "Return only your summary and nothing else. If their is no relevant content or the content is blocked, return 'no content'.\n\n"
        "Query:{query}\n\n"
        "Content:\n\n"
        "{content}"
    )

    summarizer = summary_prompt | llm


    results = []
    fetched_pages = []
    with DDGS() as ddgs:
        for r in ddgs.text(query, region="us-en", max_results=max_results):
            fetched_page = fetch_and_clean(r['href'])
            summary = summarizer.invoke({'query': query, 'content': fetched_page})
            print(summary)
            results.append(summary.content)
    return results
    
    

DuckDuckGoExtendedTool = Tool(
    name="DuckDuckGo Extended Search",
    func=duckduckgo_extended,
    description="Searches DuckDuckGo and retrieves up to N results."
)

def context_provider_node(state):
    question = state.get("question", "").lower()
    # Check if the question is related to the guided project
    try:
        context = DuckDuckGoExtendedTool.invoke(question)
        print(context)
        return {"context": context}
    except:
        print('Failed to retrieve context')
        # If unrelated or failed to retrieve context, set context to null
        return {"context": None}

In [31]:
llm = init_chat_model(model='llama3.1', model_provider='ollama')

In [32]:
def llm_qa_node(state):
    # Extract the question and context from the state
    question = state.get("question", "")
    context = state.get("context", None)

    # Check for missing context and return a fallback response
    if not context:
        return {"answer": "I don't have enough context to answer your question."}

    # Construct the prompt dynamically
    prompt = f"Context: {context}\nQuestion: {question}\nAnswer the question based on the provided context."

    try:
        response = llm.invoke(prompt)
        return {"answer": response.content.strip()}
    except Exception as e:
        return {"answer": f"An error occurred: {str(e)}"}

In [33]:
graph = StateGraph(QAState)
graph.add_node("InputNode", input_validation_node)
graph.add_node("ContextNode", context_provider_node)
graph.add_node("QANode", llm_qa_node)

graph.set_entry_point("InputNode")

graph.add_edge("InputNode", "ContextNode")
graph.add_edge("ContextNode", "QANode")
graph.add_edge("QANode", END)

qa_agent = graph.compile()

In [34]:
qa_agent.invoke({"question": "What is LangGraph?"})

content='LangGraph is a low-level orchestration framework for building, managing, and deploying long-running, stateful agents. It provides a set of prebuilt components and tools to create custom agent workflows with customizable architecture, long-term memory, and complex task handling. LangGraph offers several core benefits, including durable execution, human-in-the-loop interaction, comprehensive memory, and debugging capabilities through LangSmith. The framework can be used standalone or integrated with other LangChain products for building agents. Its ecosystem includes tools such as LangSmith for agent evaluation and observability, and the LangGraph Platform for deploying and scaling agents.' additional_kwargs={} response_metadata={'model': 'llama3.1', 'created_at': '2025-07-30T20:54:17.657874Z', 'done': True, 'done_reason': 'stop', 'total_duration': 12260521666, 'load_duration': 63538250, 'prompt_eval_count': 1173, 'prompt_eval_duration': 6334937708, 'eval_count': 122, 'eval_dura

{'question': 'What is LangGraph?',
 'context': ['LangGraph is a low-level orchestration framework for building, managing, and deploying long-running, stateful agents. It provides a set of prebuilt components and tools to create custom agent workflows with customizable architecture, long-term memory, and complex task handling. LangGraph offers several core benefits, including durable execution, human-in-the-loop interaction, comprehensive memory, and debugging capabilities through LangSmith. The framework can be used standalone or integrated with other LangChain products for building agents. Its ecosystem includes tools such as LangSmith for agent evaluation and observability, and the LangGraph Platform for deploying and scaling agents.',
  'LangGraph is a controllable cognitive architecture framework that supports diverse control flows and robustly handles complex scenarios. It enables developers to design agents that can automate real-world tasks, collaborate with humans, and persist 